## Program for image tracking. Optical flow-by Lukas Kanade optical flow

#### Use corner detection technique to identifying few points and then track them. Will use sci tomashi corner detection algorithm.

#### Idea is to detect 10 corners at the start of the video and start track them using Lucas Kanade optical flow

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
corner_detection_params = dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)

lk_params = dict(winSize=(200,200),maxLevel =2 , criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [20]:
## Read video file in OpenCV

cap = cv2.VideoCapture(0)   ## O is given for live streaming

ret,prev_frame = cap.read()

## Conversion to gray scale
prev_frame_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

## Points to track
## Use si-Tomashi corner detection GoodFeatures to Track

pts_track = cv2.goodFeaturesToTrack(prev_frame_gray,mask=None,**corner_detection_params)

mask = np.zeros_like(prev_frame)

## Use while loop for track onwards
while True:
    ret,frame = cap.read()

    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    ## Tracking
    nextpts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame_gray,frame_gray,pts_track,None,**lk_params)
    # print(mask.shape,pts_track.shape,nextpts.shape)
    
    good_new  = nextpts[status == 1]    # a complex vector whose length will be equal to maxCorners (=10)
    good_prev = pts_track[status == 1]

    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        x_new  , y_new  = new.ravel()
        x_prev , y_prev = prev.ravel()

        # print(x_new  , y_new)
        mask  = cv2.line(mask,pt1=(int(x_new),int(y_new)),pt2=(int(x_prev),int(y_prev)),color=(0,255,0),thickness=3) ## Mask is a blank video 
                                                                                                                        # frame which will use to draw connecting lines between consecutive frames 
        frame = cv2.circle(frame,(int(x_new),int(y_new)),8,(0,0,255),-1)  ## Frame is the video frame with drawn circles upon it which are directed to track.

        # print(new.ravel(),new.ravel().shape )
    # print(good_new.shape)
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)

    k = cv2.waitKey(30) & 0xFF

    if k == 27:
        break

    prev_frame_gray = frame_gray.copy()
    pts_track = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

In [18]:
cap.release()